In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup


targets = ['clear shampoo', 'sunsilk shampoo', 'lifebuoy', 'tresemme shampoo', 'ponds',
           'glow lovely', 'vaseline', 'pepsodent',
           'closeup', 'lifebuoy sabun mandi', 'lux botanicals', 'rexona', 'axe', 'molto', 'sunlight',
           'wipol', 'vixal', 'royco', 'bango kecap', 'sariwangi',  'buavita',
           'head shoulders', 'pantene shampoo', 'zinc', 'garnier', 'nivea',
           'marina', 'ciptadent pasta gigi', 'nuvo', 'giv', 'posh men body spray',
           'soklin', 'downy pelembut', 'garnier'
           'mama lemon', 'supersol', 'yuri porstex', 'masako',
           'sedaap kecap', 'abc kecap manis', 'sosro teh', 'sosro teh asli', 'tong tji',
           'country choice', 'citra', 'superpel','rinso', 'rejoice', 'posh']

locations ={"Cibubur":21, "Puri Indah": 6, "East Coast Surabaya": 32, "Metropolish": 198, "Gading Serpong": 8}


In [2]:
# Function to convert price string into a float/int
def convert_price(price_str):
    # Remove "Rp" and "." then convert to int
    return int(price_str.replace("Rp", "").replace(".", "").strip())

In [3]:
import requests
from bs4 import BeautifulSoup

# Define the URL
url = "https://shop.hypermart.co.id/hypermart/"

# Create a session to manage cookies
session = requests.Session()

# Set the 'store' cookie to a new value
session.cookies.set('store', '8')  # Replace with your desired value

# Make the request
response = session.get(url)

# Parse the HTML response
soup = BeautifulSoup(response.text, 'html.parser')

# Extract the store information
store_element = soup.select_one('div.mscurrent span')  # Use CSS selector to find the store span
if store_element:
    store_name = store_element.get_text(strip=True)
    print("Current Store:", store_name)
else:
    print("Store information not found.")

# Check if the store matches your expected value
expected_store = "Jakarta Pusat - HPM Gajah Mada"  # Replace with the expected store name
if store_name == expected_store:
    print("Store is as expected.")
else:
    print("Store has changed!")


Current Store: Tangerang - HPM Gading Serpong
Store has changed!


In [4]:
promo_sku = pd.DataFrame(columns=["productName", "basePrice", "finalPrice", "location"])


for location_name, store_id in locations.items():
    session = requests.Session()

    # Set the 'store' cookie to the store ID for the location
    session.cookies.set('store', str(store_id))  # Convert to string for compatibility
    for i in targets:

        response = session.post(f'https://shop.hypermart.co.id/hypermart/product-list.php?q={i}&sz=80')
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all product containers
        products = soup.find_all('div', class_='col')

        # List to hold the product data for this location
        products_data = []

        # Iterate over each product
        for product in products:
            # Extract the product name (inside <h5> tag)
            name = product.find('h5').text.strip()

            # Extract the price details (inside <p> tag)
            price_tag = product.find('p')

            if price_tag:
                # Check if there's a span (indicates original price)
                span_tag = price_tag.find('span')

                if span_tag and span_tag.text.strip():
                    # Original price is inside span
                    original_price = convert_price(span_tag.text.strip())
                    # Convert original_price back to string for the replace operation
                    original_price_str = span_tag.text.strip()  # Use the string from span_tag
                    # Discounted price is outside the span
                    discounted_price = convert_price(price_tag.text.replace(original_price_str, '').strip())
                else:
                    # No span means the price outside is the original price
                    original_price = convert_price(price_tag.text.strip())
                    discounted_price = original_price
                products_data.append([name, original_price, discounted_price, location_name])

        # Append the data for this location to the main DataFrame
        promo_sku = pd.concat([promo_sku, pd.DataFrame(products_data, columns=["productName", "basePrice", "finalPrice", "location"])], ignore_index=True)

# Drop duplicates based on product name and location
promo_sku.drop_duplicates(subset=["productName", "location"], inplace=True)

print(promo_sku)


                                 productName basePrice finalPrice  \
0     CLEAR SHAMPO AD SUPERFRESH APPLE 160ML     36050      36050   
1     CLEAR SHP AD COCONUT OIL + RICE 300 ML     61800      38850   
2          CLEAR SHP ICM MENTHOL SEGAR 660ML     94350      94350   
3         CLEAR SHP ICE MENTHOL SEGAR 160 ML     36050      36050   
4      CLEAR MAN 3IN1 ACTIVE CLEAN BTL 160ML     35350      35350   
...                                      ...       ...        ...   
5494            REJOICE SHP ANTI FRIZZ 150ml     30900      30900   
5495        REJOICE SHP ANTI HAIR FALL 150ml     30900      30900   
5496        REJOICE SHP ANTI HAIR FALL 340ml     54600      54600   
5497          REJOICE SHP FRIZZ REPAIR 340ml     54600      54600   
5498      REJOICE COND PERFUME SMOOTH 170 ML     24590      24590   

            location  
0            Cibubur  
1            Cibubur  
2            Cibubur  
3            Cibubur  
4            Cibubur  
...              ...  
5494  Gadi

In [5]:
from datetime import datetime

file_name = f"../hypermart/HYPERMART_{datetime.now().strftime('%y%m%d')}.xlsx"
promo_sku.to_excel(file_name,index=False)

In [6]:
# promo_sku = pd.DataFrame(columns=["productName", "basePrice", "finalPrice"])
# for i in targets:
#     session = requests.Session()

#     # Set the 'store' cookie to a new value
#     session.cookies.set('store', '8')  # Replace with your desired value

#     response = session.post(f'https://shop.hypermart.co.id/hypermart/product-list.php?q={i}&sz=80')
#     print(response.cookies)
#     soup = BeautifulSoup(response.content, 'html.parser')
#     # Find all product containers
#     products = soup.find_all('div', class_='col')
#     # List to hold the product data
#     products_data = []
#     # Iterate over each product
#     for product in products:
#         # Extract the product name (inside <h5> tag)
#         name = product.find('h5').text.strip()

#         # Extract the price details (inside <p> tag)
#         price_tag = product.find('p')

#         if price_tag:
#             # Check if there's a span (indicates original price)
#             span_tag = price_tag.find('span')

#             if span_tag and span_tag.text.strip():
#                 # Original price is inside span
#                 original_price = convert_price(span_tag.text.strip())
#                 # Convert original_price back to string for the replace operation
#                 original_price_str = span_tag.text.strip()  # Use the string from span_tag
#                 # Discounted price is outside the span
#                 discounted_price = convert_price(price_tag.text.replace(original_price_str, '').strip())
#             else:
#                 # No span means the price outside is the original price
#                 original_price = convert_price(price_tag.text.strip())
#                 discounted_price = original_price
#             products_data.append([name, original_price, discounted_price])
#     promo_sku = pd.concat([promo_sku, pd.DataFrame(products_data, columns=["productName", "basePrice", "finalPrice"])], ignore_index=True)
#     promo_sku.drop_duplicates(subset=["productName"], inplace=True)
# print(promo_sku)